In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [73]:
#Reference

# https://www.kaggle.com/scr0ll0/august-2021-lgbm-xgb-catboost-optuna#3.-Optuna

# Import packages and dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

np.random.seed(42)

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [71]:
from pandas_profiling import ProfileReport

import xgboost as xgb
import catboost as cb
import lightgbm as lgbm

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING) #Turn off printing result of each trial

In [50]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [51]:
# Create sample to build model faster
train_sample = train.sample(5000)
train_sample =train_sample.reset_index(drop=True) # Need this otherwise function below to rung XGB, Catboost etc. will throw error

In [52]:
X = train_sample.drop(['id','loss'], axis=1)
y = train_sample['loss']

test=test.drop('id',axis=1)

# Exploratory Data Analysis

In [10]:
train.shape
train.info()
train.head()
train.describe()

(2500, 102)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 38683 to 164655
Columns: 102 entries, id to loss
dtypes: float64(95), int64(7)
memory usage: 2.0 MB


,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
38683,38683,0.350894,11,0.532066,-0.846378,351.1160,1.740850,131.19900,-0.175430,174.29400,...,13028.1000,8.263590,1.60372,-0.515793,-1.062870,2.29640,0.682706,3.73659,26.66510,0
64939,64939,0.694336,60,1.380260,0.621514,48.6641,7.469840,106.38000,0.681666,-3.56084,...,140.3980,-0.377886,4.21188,0.135265,-5.683290,2.37999,0.249169,1.62031,15.48560,7
3954,3954,0.310044,22,0.870989,-0.872505,4165.5300,5.981530,302.96300,0.814370,192.29200,...,254.2760,18.159200,1.69890,0.538787,-0.082383,3.28431,0.497846,1.42185,10.36970,34
120374,120374,0.449736,23,-3.320080,0.535662,221.8440,0.511201,7.66738,0.878341,-66.60980,...,17143.8000,16.709300,3.44528,-0.346934,3.745090,2.19290,0.760123,1.71416,5.37716,1
172861,172861,0.609567,33,1.069800,1.162900,18938.5000,6.765720,196.47800,0.752956,10.19830,...,-42.3153,6.638530,3.84949,-1.148750,-18.826200,1.54535,0.293714,1.83411,2.44546,6


,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,...,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,126579.555600,0.521453,52.161600,0.085966,0.058829,3419.413315,8.131364,160.368434,0.382309,15.474679,...,4795.256863,22.420597,2.012658,0.095598,1.502388,2.439185,0.531543,1.573118,7.996887,6.580400
std,72313.415438,0.305484,42.808641,1.302198,0.806132,5850.392008,5.512764,178.574278,0.813444,99.379327,...,8419.511411,15.035164,0.913562,0.575928,9.142102,0.901719,0.229336,0.643692,5.591727,7.971559
min,41.000000,-0.059953,-5.000000,-7.492730,-1.451960,-5387.890000,0.131231,-4.734560,-3.674180,-473.851000,...,-8413.350000,-3.290380,0.162648,-1.760990,-22.505600,-0.908054,0.036861,-0.453086,0.131054,0.000000
25%,63355.000000,0.272154,19.000000,-0.604017,-0.729756,158.258250,4.026930,27.426775,-0.084394,-16.885050,...,79.834800,11.067300,1.461367,-0.400985,-3.847392,1.927817,0.346025,1.215358,3.582670,0.000000
50%,129219.000000,0.529603,41.000000,0.237091,0.007347,888.097000,7.320570,87.174150,0.633734,9.225590,...,1069.635000,19.538450,1.645585,0.256681,0.743705,2.363025,0.523195,1.452795,7.101720,3.000000
75%,190131.250000,0.785312,77.000000,0.769558,0.787415,4025.495000,10.982375,237.565250,0.927453,56.045650,...,5606.340000,32.083200,2.239897,0.517823,5.824067,2.939925,0.711336,1.889768,10.972175,10.000000
max,249923.000000,1.053830,262.000000,9.014810,1.587700,34689.800000,31.312000,898.073000,3.903630,396.940000,...,49205.000000,77.493700,4.971240,1.786750,37.960800,5.309090,1.070980,4.462110,32.941900,38.000000


# Xgboost

In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

In [8]:
params = {'n_estimators':5000,
          'learning_rate': 0.02,
          'subsample': 0.5,
          'colsample_bytree': 0.7,
          'max_depth': 6,
          'booster': 'gbtree',
#          'tree_method': 'gpu_hist', #computer with GPU
          'tree_method': 'hist',
          'reg_lambda': 60,
          'reg_alpha': 60,
           'n_jobs': 4}

In [9]:
splits = 12
stf = StratifiedKFold(n_splits=splits, shuffle=True)
oof= np.zeros((X.shape[0],))
prediction = 0
model_fi = 0
total_mean_rmse = 0

for num, (train_id, valid_id) in enumerate(stf.split(X, y)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="rmse",verbose=0)
    
    prediction += model.predict(test) / splits
    oof[valid_id] = model.predict(X_valid)
    oof[oof < 0] = 0

    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof[valid_id]))
    print(f"Fold {num} RMSE: {fold_rmse}")

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=60, reg_lambda=60, scale_pos_weight=1, subsample=0.5,
             tree_method='hist', validate_parameters=1, verbosity=None)

Fold 0 RMSE: 8.238196956516251


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=60, reg_lambda=60, scale_pos_weight=1, subsample=0.5,
             tree_method='hist', validate_parameters=1, verbosity=None)

Fold 1 RMSE: 8.413958174294343


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=60, reg_lambda=60, scale_pos_weight=1, subsample=0.5,
             tree_method='hist', validate_parameters=1, verbosity=None)

...

Fold 9 RMSE: 8.026357516026888


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=60, reg_lambda=60, scale_pos_weight=1, subsample=0.5,
             tree_method='hist', validate_parameters=1, verbosity=None)

Fold 10 RMSE: 8.170313943206805


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.02, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=60, reg_lambda=60, scale_pos_weight=1, subsample=0.5,
             tree_method='hist', validate_parameters=1, verbosity=None)

Fold 11 RMSE: 8.263423666542792


In [10]:
model.feature_importances_

array([0.00884527, 0.01007464, 0.00925265, 0.00955729, 0.01093287,
       0.0110935 , 0.00944508, 0.01059669, 0.0112201 , 0.01066267,
       0.00990177, 0.00933696, 0.00916474, 0.00947918, 0.00977434,
       0.00978735, 0.00903223, 0.01031093, 0.01001647, 0.01005229,
       0.00883139, 0.01063564, 0.00927207, 0.01033503, 0.00950968,
       0.01093784, 0.00913269, 0.01024516, 0.0097222 , 0.0101374 ,
       0.01011799, 0.01040216, 0.01092633, 0.00979137, 0.00963676,
       0.01023974, 0.01058685, 0.00872958, 0.01066628, 0.01084831,
       0.01005271, 0.01138068, 0.00915878, 0.00973155, 0.01050482,
       0.00861244, 0.01127172, 0.01082947, 0.01026849, 0.00947317,
       0.010302  , 0.00989174, 0.01139243, 0.01058205, 0.00916382,
       0.0095823 , 0.00986346, 0.00996186, 0.00935849, 0.00921139,
       0.01057919, 0.00999042, 0.00906935, 0.00991227, 0.00969541,
       0.01105355, 0.00990789, 0.00923144, 0.01069102, 0.01118085,
       0.00995666, 0.00963375, 0.01100758, 0.01028461, 0.01144

## Tune with optuna

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [59]:
def objective(trial, data =X, target =y):
    
    param = {
            'n_estimators': trial.suggest_int('n_estimators', low =400, high =4000, step =400),    
            'max_depth': trial.suggest_int('max_depth', 6, 12),
            'learning_rate': trial.suggest_float('learning_rate ', low =0.007, high =0.02),
            'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),
            'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
            'reg_alpha': trial.suggest_int('reg_alpha', 0, 50),
            'reg_lambda': trial.suggest_int('reg_lambda', 1, 50),
            'n_jobs': 4,
#            'tree_method': 'gpu_hist', #only use with GPU
#            'predictor': 'gpu_predictor', #only use with GPU
            'tree_method': 'hist',
        }

# Use cross_val_score for cross validation
#    splits = 12
#    stf = StratifiedKFold(n_splits=splits, shuffle=True)
#    model = XGBRegressor(**param)
#    mse = cross_val_score(model, X, y, cv= stf, scoring ="neg_root_mean_squared_error")
#    return mse.mean()    


    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)

    model = xgb.XGBRegressor(**param)   
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)],
              early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    return rmse

IndentationError: unexpected indent (<ipython-input-59-479e8e113975>, line 26)

In [58]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
study.best_trial
study.best_trial.values
study.best_trial.params

[I 2021-08-21 14:34:13,919] A new study created in memory with name: no-name-bd6babff-99b0-4a11-b8b7-a3accb9ba1db
[I 2021-08-21 14:34:25,995] Trial 0 finished with value: 7.571627228120726 and parameters: {'n_estimators': 400, 'max_depth': 12, 'learning_rate ': 0.016188153878399867, 'min_child_weight': 11, 'subsample': 0.9, 'colsample_bytree': 0.6, 'reg_alpha': 24, 'reg_lambda': 36}. Best is trial 0 with value: 7.571627228120726.
[I 2021-08-21 14:34:31,165] Trial 1 finished with value: 7.564092100896768 and parameters: {'n_estimators': 3200, 'max_depth': 8, 'learning_rate ': 0.018939976407824872, 'min_child_weight': 19, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 4, 'reg_lambda': 10}. Best is trial 1 with value: 7.564092100896768.
[I 2021-08-21 14:34:42,290] Trial 2 finished with value: 7.584542123686795 and parameters: {'n_estimators': 3200, 'max_depth': 10, 'learning_rate ': 0.01925638557588044, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 0.5, 'reg_alpha'

KeyboardInterrupt: 

In [ ]:
study.trials_dataframe()

In [ ]:
#plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

#plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
# went and which parts of the space were explored more
optuna.visualization.plot_slice(study)

#plot_contour: plots parameter interactions on an interactive chart. 
# You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial= {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7,'n_estimators': 4000, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 2020, 'min_child_weight': 257,'tree_method':'gpu_hist'}


preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1


np.mean(rmse)

# Catboost

In [11]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

In [12]:
params = {'depth' :6,
          'iterations' :1600,
          'learning_rate' :0.024,
          'l2_leaf_reg' :20,
          'random_strength' :1.5,
          'grow_policy' :'Depthwise',
          'leaf_estimation_method' :'Newton', 
          'bootstrap_type' :'Bernoulli',
          'thread_count' :4,
          'verbose' :False,
          'loss_function' :'RMSE', 
          'eval_metric' :'RMSE', 
          'od_type' :'Iter', 
#          'task_type' :'GPU', 
          'early_stopping_rounds' :500
}

In [13]:
splits = 12
stf = StratifiedKFold(n_splits=splits, shuffle=True)
oof= np.zeros((X.shape[0],))
prediction = 0
model_fi = 0
total_mean_rmse = 0

for num, (train_id, valid_id) in enumerate(stf.split(X, y)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=0)
    
    prediction += model.predict(test) / splits
    oof[valid_id] = model.predict(X_valid)
    oof[oof < 0] = 0

    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof[valid_id]))
    print(f"Fold {num} RMSE: {fold_rmse}")

Fold 0 RMSE: 8.011110577929017


Fold 1 RMSE: 8.002351996054163


Fold 2 RMSE: 7.983278244432914


Fold 3 RMSE: 7.922868178477842


Fold 4 RMSE: 8.019124820164457


...

Fold 7 RMSE: 8.04306409158023


Fold 8 RMSE: 7.930784375603254


Fold 9 RMSE: 7.928860464839702


Fold 10 RMSE: 7.981307247426467


Fold 11 RMSE: 7.9141092615302915


## Tune with optuna

In [64]:
def objective(trial, data =X, target =y):
    
    param = {
            'max_depth': trial.suggest_int('max_depth', 6, 12),
            'learning_rate': trial.suggest_float('eta', 0.007, 0.013),
            'n_estimators': trial.suggest_int('n_estimators', 500, 25000, 500),
            'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
            'reg_lambda': trial.suggest_int('reg_lambda', 1, 50),
#            'task_type': 'GPU', # only with GPU
            'task_type': 'CPU',
            'bootstrap_type': 'Bernoulli', 
            'eval_metric':'RMSE',
            'verbose': False
        }

    
# Use cross_val_score for cross validation
#    splits = 12
#    stf = StratifiedKFold(n_splits=splits, shuffle=True)
#    model = CatBoostRegressor(**param)
#    mse = cross_val_score(model, X, y, cv= stf, scoring ="neg_root_mean_squared_error")
#    return mse.mean()    


    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)

    model = CatBoostRegressor(**param)   
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)],
              early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    return rmse

In [65]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
study.best_trial
study.best_trial.values
study.best_trial.params

[I 2021-08-21 15:00:43,138] A new study created in memory with name: no-name-cffe3d3e-93f9-4943-a239-fafdeb30fbc9
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2021-08-21 15:03:37,780] Trial 0 finished with value: 7.5682322060148985 and parameters: {'max_depth': 9, 'eta': 0.012880894489084875, 'n_estimators': 3000, 'subsample': 0.5, 'reg_lambda': 45}. Best is trial 0 with value: 7.5682322060148985.


KeyboardInterrupt: 

In [ ]:
#plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

#plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
# went and which parts of the space were explored more
optuna.visualization.plot_slice(study)

#plot_contour: plots parameter interactions on an interactive chart. 
# You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial = {'l2_leaf_reg': 0.02247766515106271, 'max_bin': 364, 'subsample': 0.6708650091202213,
             'learning_rate': 0.010290546311954876, 'max_depth': 10, 'random_state': 24, 'min_data_in_leaf': 300,
            'task_type': 'GPU','loss_function': 'RMSE','n_estimators':  25000}


preds = np.zeros(test.shape[0])
kf = KFold(n_splits=10,random_state=48,shuffle=True)
rmse=[]   # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = CatBoostRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1

    
np.mean(rmse)

# LightGBM

In [14]:
from lightgbm import LGBMRegressor

lgb =LGBMRegressor()

In [17]:
splits = 12
stf = StratifiedKFold(n_splits=splits, shuffle=True)
oof= np.zeros((X.shape[0],))
prediction = 0
model_fi = 0
total_mean_rmse = 0

for num, (train_id, valid_id) in enumerate(stf.split(X, y)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = LGBMRegressor()
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=0)
    
    prediction += model.predict(test) / splits
    oof[valid_id] = model.predict(X_valid)
    oof[oof < 0] = 0

    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof[valid_id]))
    print(f"Fold {num} RMSE: {fold_rmse}")

LGBMRegressor()

Fold 0 RMSE: 8.196659741320799


LGBMRegressor()

Fold 1 RMSE: 8.23003072965846


LGBMRegressor()

...

Fold 9 RMSE: 8.015337557205292


LGBMRegressor()

Fold 10 RMSE: 8.323322036161233


LGBMRegressor()

Fold 11 RMSE: 8.164025454610773


## Tune with optuna

In [87]:
def objective(trial, data =X, target =y):
    
    param = {
            'boosting_type': 'goss',
            'max_depth': trial.suggest_int('max_depth', 6, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.007, 0.013),
            'n_estimators': trial.suggest_int('n_estimators', 400, 4000, 400),
            'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),
            'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
            'reg_alpha': trial.suggest_int('reg_alpha', 0, 50),
            'reg_lambda': trial.suggest_int('reg_lambda', 1, 50),
            'n_jobs': 4,
        }

# Use cross_val_score for cross validation
#    splits = 12
#    stf = StratifiedKFold(n_splits=splits, shuffle=True)
#    model = XGBRegressor(**param)
#    mse = cross_val_score(model, X, y, cv= stf, scoring ="neg_root_mean_squared_error")
#    return mse.mean()    


    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)

    model = LGBMRegressor(**param)   
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)],
              early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    return rmse

In [88]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=2)
print('Number of finished trials:', len(study.trials))
study.best_trial
study.best_trial.values
study.best_trial.params

Number of finished trials: 2


FrozenTrial(number=0, values=[7.587008597765351], datetime_start=datetime.datetime(2021, 8, 21, 16, 14, 1, 6318), datetime_complete=datetime.datetime(2021, 8, 21, 16, 14, 2, 738649), params={'max_depth': 7, 'learning_rate': 0.012471233558211208, 'n_estimators': 1200, 'min_child_weight': 13, 'subsample': 0.7, 'colsample_bytree': 0.9, 'reg_alpha': 39, 'reg_lambda': 25}, distributions={'max_depth': IntUniformDistribution(high=12, low=6, step=1), 'learning_rate': UniformDistribution(high=0.013, low=0.007), 'n_estimators': IntUniformDistribution(high=4000, low=400, step=400), 'min_child_weight': IntUniformDistribution(high=20, low=5, step=1), 'subsample': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 'colsample_bytree': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 'reg_alpha': IntUniformDistribution(high=50, low=0, step=1), 'reg_lambda': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.CO

[7.587008597765351]

{'max_depth': 7,
 'learning_rate': 0.012471233558211208,
 'n_estimators': 1200,
 'min_child_weight': 13,
 'subsample': 0.7,
 'colsample_bytree': 0.9,
 'reg_alpha': 39,
 'reg_lambda': 25}

In [70]:
study.trials_dataframe()

TypeError: argument of type 'int' is not iterable

In [72]:
#plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

#plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
# went and which parts of the space were explored more
optuna.visualization.plot_slice(study)

#plot_contour: plots parameter interactions on an interactive chart. 
# You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [90]:
Best_trial= study.best_trial.params


preds = np.zeros(test.shape[0])
kf = StratifiedKFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1


np.mean(rmse)

NameError: name 'columns' is not defined

# Google AutoML

# Azure AutoML

# H2O AutoML

In [ ]:
https://www.kaggle.com/keenborder/h20-automl-vs-boosting-models

In [24]:
import h2o
from h2o.automl import H2OAutoML

In [26]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.6+8-b765.1, mixed mode)
  Starting server from C:\Users\Frank\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Frank\AppData\Local\Temp\tmpnfa_reah
  JVM stdout: C:\Users\Frank\AppData\Local\Temp\tmpnfa_reah\h2o_Frank_started_from_python.out
  JVM stderr: C:\Users\Frank\AppData\Local\Temp\tmpnfa_reah\h2o_Frank_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_Frank_ua5c63
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.967 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [27]:
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
x = train.columns
y = "loss"
x.remove(y)
x.remove("id")

In [29]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1,max_runtime_secs = 600)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
12:13:41.486: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [31]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210821_121341,62.0613,7.8779,62.0613,6.13602,1.21911
StackedEnsemble_BestOfFamily_AutoML_20210821_121341,62.0829,7.87927,62.0829,6.13723,1.21944
GLM_1_AutoML_20210821_121341,62.3718,7.89758,62.3718,6.15479,1.22392
GBM_grid__1_AutoML_20210821_121341_model_1,62.4021,7.8995,62.4021,6.16299,1.22616
GBM_2_AutoML_20210821_121341,62.6627,7.91598,62.6627,6.17674,1.22911
GBM_1_AutoML_20210821_121341,62.6875,7.91754,62.6875,6.17848,1.22942
GBM_3_AutoML_20210821_121341,62.6956,7.91806,62.6956,6.17842,1.22937
GBM_4_AutoML_20210821_121341,62.7278,7.92009,62.7278,6.17915,1.22946
DeepLearning_1_AutoML_20210821_121341,62.743,7.92105,62.743,6.10688,1.21273
GBM_5_AutoML_20210821_121341,62.7709,7.92281,62.7709,6.18116,1.22987


In [30]:
preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
# Convert submission to Dataframe, then to csv
submission=pd.DataFrame({
    "id": h2o.as_list(test["id"], use_pandas=True).values[:,0],
    "loss": h2o.as_list(preds, use_pandas=True).values[:,0]
    
})

submission.to_csv("submission.csv",index=False)

# Sklearn Ensemble

In [ ]:
from sklearn.ensemble import VotingRegressor